In [1]:
import torch
import numpy as np
import pickle

from vocab import Vocab, load_vocab
from common import Data, Split, Batches, load_data, encode_y, load_split
from utils import ProgressBar, evaluate
from pack import Pack
from model import Model, Model2, TextOnlyModel
from train import train_epoches

%load_ext autoreload
%autoreload 2

In [2]:
genre_list = pickle.load(open("../data/tmdb_genres_list.pkl", 'rb'))

GENRES = load_data("../local/genres.pkl")
train = load_split("../local/train.pkl")
val = load_split("../local/val.pkl")
test = load_split("../local/test.pkl")
embedding = torch.load('../local/embedding.pth')
OVERVIEWS_ENCODED = load_data("../local/overviews_encoded.pkl")
TITLES_ENCODED = load_data("../local/titles_encoded.pkl")

In [3]:
_model = Model2(embedding, hidden_dim=512, num_layers=3, cuda=True)
_model.encoder = torch.load("./saved/encoder_500.pth")
_model.classifier = torch.load("./saved/rnn_cls_500.pth")
_model.cuda()

for param in _model.encoder.parameters():
    param.requires_grad = False
    
optimizer = torch.optim.Adam(filter(lambda p:p.requires_grad, _model.classifier.parameters()))
loss = torch.nn.BCEWithLogitsLoss().cuda()

model = TextOnlyModel(_model.encoder, _model.classifier, OVERVIEWS_ENCODED, GENRES)

In [4]:
evaluate(train, model, 128)
evaluate(val, model, 128)

P 0.9673742742158459 	R: 0.9730539847347115 	F1: 0.9702058171062362
P 0.456787109375 	R: 0.5251192815043503 	F1: 0.48857553205379295


(0.456787109375, 0.5251192815043503, 0.48857553205379295)

In [10]:
losses = train_epoches(n_epochs=1, model=model, train=train, loss=loss, val=val,
              batch_size=128, optimizer=optimizer, scheduler=None)

...5..10...15..20..25...30..35..40...45..50..55...60..65..70...75..80..85...90..95..100 F
epoch 0: 0.0091985995
Train:	P 0.9777579797855672 	R: 0.9835290482076637 	F1: 0.9806350233396497
Val:	P 0.455810546875 	R: 0.5251758087201125 	F1: 0.48804077898313947


In [ ]:
evaluate(test, model)